In [ ]:
# 참고한 사이트입니다.

# hankuk_geo.json
# 대한민국 행정동 경계 파일입니다.

#     통계청 통계지리정보서비스에서 제공하는 행정동 경계 파일을 바탕으로, 행정동 변경 이력을 수정하여 반영한 파일입니다.
#     geojson 형식이며, 좌표계는 WGS84 (EPSG:4326) 입니다.
#     geojson 및 csv 파일 인코딩 형식은 UTF-8입니다.
#     topology 정합성을 검토하였습니다.(폴리곤 경계끼리 잘 맞습니다)
#     속성의 adm_cd 는 통계청에서 사용하는 7자리의 [한국행정구역분류코드]입니다.
#     속성의 adm_cd2 는 행정안전부 사용하는 10자리의 [행정기관코드]입니다.(2018.07.24 업데이트 파일부터 적용)
#     속성의 adm_nm 은 통계청에서 사용하는 전국 행정동 이름입니다.

#https://github.com/vuski/admdongkor

# ===================================================================================================================

# 지도 데이터 시각화
# https://dailyheumsi.tistory.com/141 --> shp 파일을 pandas 에서 시각화

# https://blog.naver.com/kcchang61/221350672356


# ===================================================================================================================

# 데이터 전처리
# https://note.espriter.net/1325 -> 특정 값 컬럼 제거

# 특정 범위 내 마커 파악
# https://devtalk.kakao.com/t/circle/106394/2

# 마커 클러스터
# https://oboki.net/workspace/python/folium-%EC%A7%80%EB%A6%AC%EC%A0%95%EB%B3%B4-%EC%8B%9C%EA%B0%81%ED%99%94/

In [ ]:
import pandas as pd
import json
import folium
import matplotlib


# https://blog.naver.com/kcchang61/221350672356

# 구단위 지도 그리기
geo_path = './data/pro_data/hankuk1_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8')) # 딕셔너리 객체로 리턴

# 서울 지하철역 정보
subway_path = './data/pro_data/서울교통공사 지하철역 주소 및 전화번호 정보.json'
sub_str = json.load(open(subway_path, encoding='utf-8'))


# # 지역별, 동별 구분 경계선 
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
fmap=folium.Choropleth(geo_data = geo_str,
#                        color = 'Blues',
                       key_on='feature.properties.adm_nm').add_to(map) # 키온에 지역 명을 넣어주는 것
fmap.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'],labels=False) # 해당 지역마다 툴팁으로 마우스오버 작동하게 만듦
)


# # 지하철 리스트를 데이터프레임 변환
df = pd.read_excel('./data/pro_data/지하철노선위경도정보1.xlsx') # 데이터 프레임 형성

# # 서울 지하철 리스트 마커로 표현하기
for sub, lat, lng in zip(df.역이름, df.위도, df.경도): # zip으로 수행 / 하나씩 묶는 것 
    folium.Marker([lat, lng], popup=sub).add_to(map) # 마커 객체 생성 // 팝업 시 이름이 나오도록 설정 / 서울 지도에 넣는다.
display(map)


In [48]:
## 군집분석을 시도해보자!!

import pandas as pd
import seaborn as sns
import numpy as np

# 군집분석용 데이터 불러오기
knn_data = pd.read_excel('./data/pro_data/last.xlsx')
knn_data = knn_data.drop(knn_data.columns[[0]], axis=1)
# display(knn_data.head())

# 군집분석 전 NaN 값 0으로 치환하기
knn_data.fillna(0, inplace=True)
# knn_data.info()

# 자전거 이용 비율을 1을 기준으로 넘지 못하면 0, 넘으면 1로 잡자 --> 연속형 데이터를 범주형으로 

# 1. cut()이용해 자전거이용비율을 구간분할 한다
boundary = np.histogram(knn_data['자전거이용비율'], bins=3)

col_name = ['저이용', '보통이용', '고이용']
all_of_data['자전거이용비율분류'] = pd.cut(x=all_of_data['자전거이용비율'],
                                          bins = boundary,
                                          labels=col_name,
                                          include_lowest = True)
print(all_of_data[['자전거이용비율', '자전거이용비율분류']])

# 2. 원핫인코딩으로 더미변수를 만들어 둔다. -> 자전거이용비율컬럼은 삭제한다.
# 3. knn 분석을 시행한다.

# 범주형 데이터를 가지고 

ValueError: setting an array element with a sequence.

In [ ]:
# 여기서부터는 사전에 처리한 전처리 코드입니당!!

In [ ]:
# geojson에서 특정 값 추출하기!!
import pandas as pd
# json 파일 불러오기
import json
geo_path = './data/pro_data/hankuk1_geo.json'
geo_str = json.load(open(geo_path, encoding='utf-8')) 

# 행정동 추출하기
geo_nm = []
for i in range(425):
    points = geo_str['features'][i]['properties']['adm_nm']
#     print(points)
    geo_nm.append(points)
geo_data = pd.DataFrame(geo_nm)
display(geo_data)
# geo_data.to_excel('./data/동정보만.xlsx')

# 행정동 경계 추출하기
geo_loc = []
for i in range(425):
    points = geo_str['features'][i]['geometry']['coordinates']
    geo_loc.append(points)
    
geo_data1 =pd.DataFrame(geo_loc)
display(geo_data1)

df_all = pd.concat([geo_data, geo_data1], axis=1)
df_all.columns = ['행정동', '경계']
df_all.to_excel('./data/경계.xlsx')
display(df_all)

In [ ]:
# 행정동 전처리 -> geojson과 동일하게 이름을 바꾼 뒤 merge로 합치기 위한 사전작업
import pandas as pd
pop = pd.read_csv('./data/pro_data/서울동별인구데이터1.txt', sep="\t")
# 서울 동별 인구 "합계", "소계" 없애기
pop_1 = pop['자치구'].isin(['합계','소계'])
pop_2 = pop[~pop_1]
pop_3 = pop_2['동'].isin(['합계','소계'])
pop_last = pop_2[~pop_3]

pop_last.to_csv('./data/pro_data/서울동별인구데이터1.csv')

pop = pd.read_csv('./data/pro_data/서울동별인구데이터1.csv')
pop.drop('Unnamed: 0',axis=1, inplace=True)
display(pop)

# 새로운 동 이름 만들어보기

list1 = []
for i in range(424):
    x = '서울특별시'
    y = pop.loc[i]['자치구']
    z = pop.loc[i]['동']
    a = x +' '+ y +' '+ z
    list1.append(a)
    print('"',a,'"', sep="", end=", ")

pop['주소'] = list1

geo_data1['주소'] = ["서울특별시 종로구 사직동", "서울특별시 종로구 삼청동", "서울특별시 종로구 부암동", "서울특별시 종로구 평창동", "서울특별시 종로구 무악동", "서울특별시 종로구 교남동", "서울특별시 종로구 가회동", "서울특별시 종로구 종로1·2·3·4가동", "서울특별시 종로구 종로5·6가동", "서울특별시 종로구 이화동", "서울특별시 종로구 창신1동", "서울특별시 종로구 창신2동", "서울특별시 종로구 창신3동", "서울특별시 종로구 숭인1동", "서울특별시 종로구 숭인2동", "서울특별시 종로구 청운효자동", "서울특별시 종로구 혜화동", "서울특별시 중구 소공동", "서울특별시 중구 회현동", "서울특별시 중구 명동", "서울특별시 중구 필동", "서울특별시 중구 장충동", "서울특별시 중구 광희동", "서울특별시 중구 을지로동", "서울특별시 중구 신당5동", "서울특별시 중구 황학동", "서울특별시 중구 중림동", "서울특별시 중구 신당동", "서울특별시 중구 다산동", "서울특별시 중구 약수동", "서울특별시 중구 청구동", "서울특별시 중구 동화동", "서울특별시 용산구 후암동", "서울특별시 용산구 용산2가동", "서울특별시 용산구 남영동", "서울특별시 용산구 원효로2동", "서울특별시 용산구 효창동", "서울특별시 용산구 용문동", "서울특별시 용산구 이촌1동", "서울특별시 용산구 이촌2동", "서울특별시 용산구 이태원1동", "서울특별시 용산구 이태원2동", "서울특별시 용산구 서빙고동", "서울특별시 용산구 보광동", "서울특별시 용산구 청파동", "서울특별시 용산구 원효로1동", "서울특별시 용산구 한강로동", "서울특별시 용산구 한남동", "서울특별시 성동구 왕십리2동", "서울특별시 성동구 마장동", "서울특별시 성동구 사근동", "서울특별시 성동구 행당1동", "서울특별시 성동구 행당2동", "서울특별시 성동구 응봉동", "서울특별시 성동구 금호1가동", "서울특별시 성동구 금호4가동", "서울특별시 성동구 성수1가1동", "서울특별시 성동구 성수1가2동", "서울특별시 성동구 성수2가1동", "서울특별시 성동구 성수2가3동", "서울특별시 성동구 송정동", "서울특별시 성동구 용답동", "서울특별시 성동구 왕십리도선동", "서울특별시 성동구 금호2·3가동", "서울특별시 성동구 옥수동", "서울특별시 광진구 화양동", "서울특별시 광진구 군자동", "서울특별시 광진구 중곡1동", "서울특별시 광진구 중곡2동", "서울특별시 광진구 중곡3동", "서울특별시 광진구 중곡4동", "서울특별시 광진구 능동", "서울특별시 광진구 구의1동", "서울특별시 광진구 구의2동", "서울특별시 광진구 구의3동", "서울특별시 광진구 광장동", "서울특별시 광진구 자양1동", "서울특별시 광진구 자양2동", "서울특별시 광진구 자양3동", "서울특별시 광진구 자양4동", "서울특별시 동대문구 회기동", "서울특별시 동대문구 휘경1동", "서울특별시 동대문구 휘경2동", "서울특별시 동대문구 청량리동", "서울특별시 동대문구 용신동", "서울특별시 동대문구 제기동", "서울특별시 동대문구 전농1동", "서울특별시 동대문구 전농2동", "서울특별시 동대문구 답십리1동", "서울특별시 동대문구 답십리2동", "서울특별시 동대문구 장안1동", "서울특별시 동대문구 장안2동", "서울특별시 동대문구 이문1동", "서울특별시 동대문구 이문2동", "서울특별시 중랑구 면목2동", "서울특별시 중랑구 면목4동", "서울특별시 중랑구 면목5동", "서울특별시 중랑구 면목7동", "서울특별시 중랑구 상봉1동", "서울특별시 중랑구 상봉2동", "서울특별시 중랑구 중화1동", "서울특별시 중랑구 중화2동", "서울특별시 중랑구 묵1동", "서울특별시 중랑구 묵2동", "서울특별시 중랑구 망우3동", "서울특별시 중랑구 신내1동", "서울특별시 중랑구 신내2동", "서울특별시 중랑구 면목본동", "서울특별시 중랑구 면목3·8동", "서울특별시 중랑구 망우본동", "서울특별시 성북구 돈암1동", "서울특별시 성북구 돈암2동", "서울특별시 성북구 안암동", "서울특별시 성북구 보문동", "서울특별시 성북구 정릉1동", "서울특별시 성북구 정릉2동", "서울특별시 성북구 정릉3동", "서울특별시 성북구 정릉4동", "서울특별시 성북구 길음1동", "서울특별시 성북구 길음2동", "서울특별시 성북구 월곡1동", "서울특별시 성북구 월곡2동", "서울특별시 성북구 장위1동", "서울특별시 성북구 장위2동", "서울특별시 성북구 장위3동", "서울특별시 성북구 성북동", "서울특별시 성북구 삼선동", "서울특별시 성북구 동선동", "서울특별시 성북구 종암동", "서울특별시 성북구 석관동", "서울특별시 강북구 번1동", "서울특별시 강북구 번2동", "서울특별시 강북구 번3동", "서울특별시 강북구 수유1동", "서울특별시 강북구 수유2동", "서울특별시 강북구 수유3동", "서울특별시 강북구 삼양동", "서울특별시 강북구 미아동", "서울특별시 강북구 송중동", "서울특별시 강북구 송천동", "서울특별시 강북구 삼각산동", "서울특별시 강북구 우이동", "서울특별시 강북구 인수동", "서울특별시 도봉구 쌍문1동", "서울특별시 도봉구 쌍문2동", "서울특별시 도봉구 쌍문3동", "서울특별시 도봉구 쌍문4동", "서울특별시 도봉구 방학1동", "서울특별시 도봉구 방학2동", "서울특별시 도봉구 방학3동", "서울특별시 도봉구 창1동", "서울특별시 도봉구 창2동", "서울특별시 도봉구 창3동", "서울특별시 도봉구 창4동", "서울특별시 도봉구 창5동", "서울특별시 도봉구 도봉1동", "서울특별시 도봉구 도봉2동", "서울특별시 노원구 월계1동", "서울특별시 노원구 월계2동", "서울특별시 노원구 월계3동", "서울특별시 노원구 공릉2동", "서울특별시 노원구 하계1동", "서울특별시 노원구 하계2동", "서울특별시 노원구 중계본동", "서울특별시 노원구 중계1동", "서울특별시 노원구 중계4동", "서울특별시 노원구 상계1동", "서울특별시 노원구 상계2동", "서울특별시 노원구 상계5동", "서울특별시 노원구 상계8동", "서울특별시 노원구 상계9동", "서울특별시 노원구 상계10동", "서울특별시 노원구 상계3·4동", "서울특별시 노원구 상계6·7동", "서울특별시 노원구 중계2·3동", "서울특별시 노원구 공릉1동", "서울특별시 은평구 녹번동", "서울특별시 은평구 불광1동", "서울특별시 은평구 갈현1동", "서울특별시 은평구 갈현2동", "서울특별시 은평구 구산동", "서울특별시 은평구 대조동", "서울특별시 은평구 응암1동", "서울특별시 은평구 응암2동", "서울특별시 은평구 신사1동", "서울특별시 은평구 신사2동", "서울특별시 은평구 증산동", "서울특별시 은평구 수색동", "서울특별시 은평구 진관동", "서울특별시 은평구 불광2동", "서울특별시 은평구 응암3동", "서울특별시 은평구 역촌동", "서울특별시 서대문구 천연동", "서울특별시 서대문구 홍제1동", "서울특별시 서대문구 홍제3동", "서울특별시 서대문구 홍제2동", "서울특별시 서대문구 홍은1동", "서울특별시 서대문구 홍은2동", "서울특별시 서대문구 남가좌1동", "서울특별시 서대문구 남가좌2동", "서울특별시 서대문구 북가좌1동", "서울특별시 서대문구 북가좌2동", "서울특별시 서대문구 충현동", "서울특별시 서대문구 북아현동", "서울특별시 서대문구 신촌동", "서울특별시 서대문구 연희동", "서울특별시 마포구 용강동", "서울특별시 마포구 대흥동", "서울특별시 마포구 염리동", "서울특별시 마포구 신수동", "서울특별시 마포구 서교동", "서울특별시 마포구 합정동", "서울특별시 마포구 망원1동", "서울특별시 마포구 망원2동", "서울특별시 마포구 연남동", "서울특별시 마포구 성산1동", "서울특별시 마포구 성산2동", "서울특별시 마포구 상암동", "서울특별시 마포구 도화동", "서울특별시 마포구 서강동", "서울특별시 마포구 공덕동", "서울특별시 마포구 아현동", "서울특별시 양천구 목1동", "서울특별시 양천구 목2동", "서울특별시 양천구 목3동", "서울특별시 양천구 목4동", "서울특별시 양천구 신월1동", "서울특별시 양천구 신월2동", "서울특별시 양천구 신월3동", "서울특별시 양천구 신월4동", "서울특별시 양천구 신월5동", "서울특별시 양천구 신월6동", "서울특별시 양천구 신월7동", "서울특별시 양천구 신정1동", "서울특별시 양천구 신정2동", "서울특별시 양천구 신정3동", "서울특별시 양천구 신정6동", "서울특별시 양천구 신정7동", "서울특별시 양천구 목5동", "서울특별시 양천구 신정4동", "서울특별시 강서구 염창동", "서울특별시 강서구 등촌1동", "서울특별시 강서구 등촌2동", "서울특별시 강서구 등촌3동", "서울특별시 강서구 화곡본동", "서울특별시 강서구 화곡2동", "서울특별시 강서구 화곡3동", "서울특별시 강서구 화곡4동", "서울특별시 강서구 화곡6동", "서울특별시 강서구 화곡8동", "서울특별시 강서구 가양1동", "서울특별시 강서구 가양2동", "서울특별시 강서구 가양3동", "서울특별시 강서구 발산1동", "서울특별시 강서구 공항동", "서울특별시 강서구 방화1동", "서울특별시 강서구 방화2동", "서울특별시 강서구 방화3동", "서울특별시 강서구 화곡1동", "서울특별시 강서구 우장산동", "서울특별시 구로구 신도림동", "서울특별시 구로구 구로1동", "서울특별시 구로구 구로3동", "서울특별시 구로구 구로4동", "서울특별시 구로구 구로5동", "서울특별시 구로구 고척1동", "서울특별시 구로구 고척2동", "서울특별시 구로구 개봉2동", "서울특별시 구로구 개봉3동", "서울특별시 구로구 오류1동", "서울특별시 구로구 오류2동", "서울특별시 구로구 수궁동", "서울특별시 구로구 가리봉동", "서울특별시 구로구 구로2동", "서울특별시 구로구 개봉1동", "서울특별시 금천구 가산동", "서울특별시 금천구 독산1동", "서울특별시 금천구 독산2동", "서울특별시 금천구 독산3동", "서울특별시 금천구 독산4동", "서울특별시 금천구 시흥1동", "서울특별시 금천구 시흥2동", "서울특별시 금천구 시흥3동", "서울특별시 금천구 시흥4동", "서울특별시 금천구 시흥5동", "서울특별시 영등포구 여의동", "서울특별시 영등포구 당산1동", "서울특별시 영등포구 당산2동", "서울특별시 영등포구 양평1동", "서울특별시 영등포구 양평2동", "서울특별시 영등포구 신길1동", "서울특별시 영등포구 신길3동", "서울특별시 영등포구 신길4동", "서울특별시 영등포구 신길5동", "서울특별시 영등포구 신길6동", "서울특별시 영등포구 신길7동", "서울특별시 영등포구 대림1동", "서울특별시 영등포구 대림2동", "서울특별시 영등포구 대림3동", "서울특별시 영등포구 영등포본동", "서울특별시 영등포구 영등포동", "서울특별시 영등포구 도림동", "서울특별시 영등포구 문래동", "서울특별시 동작구 노량진2동", "서울특별시 동작구 상도1동", "서울특별시 동작구 상도2동", "서울특별시 동작구 상도3동", "서울특별시 동작구 상도4동", "서울특별시 동작구 사당1동", "서울특별시 동작구 사당3동", "서울특별시 동작구 사당4동", "서울특별시 동작구 사당5동", "서울특별시 동작구 대방동", "서울특별시 동작구 신대방1동", "서울특별시 동작구 신대방2동", "서울특별시 동작구 흑석동", "서울특별시 동작구 노량진1동", "서울특별시 동작구 사당2동", "서울특별시 관악구 보라매동", "서울특별시 관악구 청림동", "서울특별시 관악구 행운동", "서울특별시 관악구 낙성대동", "서울특별시 관악구 중앙동", "서울특별시 관악구 인헌동", "서울특별시 관악구 남현동", "서울특별시 관악구 서원동", "서울특별시 관악구 신원동", "서울특별시 관악구 서림동", "서울특별시 관악구 신사동", "서울특별시 관악구 신림동", "서울특별시 관악구 난향동", "서울특별시 관악구 조원동", "서울특별시 관악구 대학동", "서울특별시 관악구 은천동", "서울특별시 관악구 성현동", "서울특별시 관악구 청룡동", "서울특별시 관악구 난곡동", "서울특별시 관악구 삼성동", "서울특별시 관악구 미성동", "서울특별시 서초구 서초1동", "서울특별시 서초구 서초2동", "서울특별시 서초구 서초3동", "서울특별시 서초구 서초4동", "서울특별시 서초구 잠원동", "서울특별시 서초구 반포본동", "서울특별시 서초구 반포1동", "서울특별시 서초구 반포2동", "서울특별시 서초구 반포3동", "서울특별시 서초구 반포4동", "서울특별시 서초구 방배본동", "서울특별시 서초구 방배1동", "서울특별시 서초구 방배2동", "서울특별시 서초구 방배3동", "서울특별시 서초구 방배4동", "서울특별시 서초구 양재1동", "서울특별시 서초구 양재2동", "서울특별시 서초구 내곡동", "서울특별시 강남구 신사동", "서울특별시 강남구 논현1동", "서울특별시 강남구 논현2동", "서울특별시 강남구 삼성1동", "서울특별시 강남구 삼성2동", "서울특별시 강남구 대치1동", "서울특별시 강남구 대치4동", "서울특별시 강남구 역삼1동", "서울특별시 강남구 역삼2동", "서울특별시 강남구 도곡1동", "서울특별시 강남구 도곡2동", "서울특별시 강남구 개포1동", "서울특별시 강남구 개포4동", "서울특별시 강남구 일원본동", "서울특별시 강남구 일원1동", "서울특별시 강남구 일원2동", "서울특별시 강남구 수서동", "서울특별시 강남구 세곡동", "서울특별시 강남구 압구정동", "서울특별시 강남구 청담동", "서울특별시 강남구 대치2동", "서울특별시 강남구 개포2동", "서울특별시 송파구 풍납1동", "서울특별시 송파구 풍납2동", "서울특별시 송파구 거여1동", "서울특별시 송파구 거여2동", "서울특별시 송파구 마천1동", "서울특별시 송파구 마천2동", "서울특별시 송파구 방이1동", "서울특별시 송파구 방이2동", "서울특별시 송파구 오륜동", "서울특별시 송파구 오금동", "서울특별시 송파구 송파1동", "서울특별시 송파구 송파2동", "서울특별시 송파구 석촌동", "서울특별시 송파구 삼전동", "서울특별시 송파구 가락본동", "서울특별시 송파구 가락1동", "서울특별시 송파구 가락2동", "서울특별시 송파구 문정1동", "서울특별시 송파구 문정2동", "서울특별시 송파구 장지동", "서울특별시 송파구 위례동", "서울특별시 송파구 잠실본동", "서울특별시 송파구 잠실4동", "서울특별시 송파구 잠실6동", "서울특별시 송파구 잠실7동", "서울특별시 송파구 잠실2동", "서울특별시 송파구 잠실3동", "서울특별시 강동구 강일동", "서울특별시 강동구 상일동", "서울특별시 강동구 명일1동", "서울특별시 강동구 명일2동", "서울특별시 강동구 고덕1동", "서울특별시 강동구 고덕2동", "서울특별시 강동구 암사2동", "서울특별시 강동구 암사3동", "서울특별시 강동구 천호1동", "서울특별시 강동구 천호3동", "서울특별시 강동구 성내1동", "서울특별시 강동구 성내2동", "서울특별시 강동구 성내3동", "서울특별시 강동구 둔촌1동", "서울특별시 강동구 둔촌2동", "서울특별시 강동구 암사1동", "서울특별시 강동구 천호2동", "서울특별시 강동구 길동"]
geo_data1.to_excel('./data/pro_data/지리정보만.xlsx')



display(pop.head(15))

In [ ]:
# geo_data와 pop을 merge로 결합하여 NaN 값 파악
import pandas as pd

final = pd.merge(geo_data, pop, how='outer', left_on='adm_nm', right_on='주소')
print(final)

final.to_excel('./data/pro_data/통합정리.xlsx') 
# 항동 누락 --> 통합정리 파일에 직접적 추가

In [ ]:
# 행정동 경계에서 지하철 개수 계산하기
import pandas as pd
import numpy as np

# 지하철 개수를 열로 만들어붙일 통합정리 파일을 불러온다.
all_of_df = pd.read_excel('./data/pro_data/통합정리1.xlsx')

# 지하철 역 정보를 불러온다. (역이름, 위도, 경도)
ditie = pd.read_excel('./data/pro_data/지하철노선위경도정보3.xlsx')
# display(ditie.head())

# 행정동 파일을 불러와 데이터프레임을 동과 경계로 나눈다. (경계는 경도, 위도로 작성됨)
boundary = pd.read_excel('./data/pro_data/경계1.xlsx')
# display(boundary.head())


# 행정동 경계를 동마다 위도 경도로 나눠서 파악할 수 있도록 나눠보기 --> 1행만 꺼내서 파악
# a = boundary.loc[1][2]
# print(a)
# aa = a.split(", ")
# print(aa)

list1 = []
# 행정동 별로 경계를 unlist해서, 위도와 경도값을 각각 나눈 뒤, 위도와 경도로 각각 묶는다.
# 모든 내용을 str로 인식해 list()로 리스트를 만들면 모든 숫자가 분리 --> split을 이용해 분류
for i in range(len(boundary)):
    a = boundary.loc[i][2] 
    aa = a.split(", ")
        
    lat = [] # 위도 리스트
    lng = [] # 경도 리스트
    for z in range(len(aa)):
        if z % 2 == 0:
            lng.append(aa[z]) # 경도
        else :
            lat.append(aa[z]) # 위도

        
# # 각 리스트의 최댓값과 최솟값을 구한다. --> str을 float()를 통해 실수로 변환
    max_lat = float(max(lat));
    min_lat = float(min(lat));
    max_lng = float(max(lng));
    min_lng = float(min(lng));
    

# # 각 리스트의 최댓값과 최솟값을 사이에 두고 지하철 역의 위도, 경도가 포함되는지 확인한다 -> 확인 되면 i+1 과 같은 형태로 리턴한다.
    i = 0
    for k in range(len(ditie)):
        if min_lat < ditie.iloc[k,1] < max_lat  :
            if min_lng < ditie.iloc[k,2] < max_lng :
                i = i + 1   
    list1.append(i)


 # 사직동 안에 지하철 2개니까 첫 원소는 2여야 함
    
# # 마지막으로 리턴된 값을 새로운 리스트(지하철 개수)에 담는다.
all_of_df['지하철개수'] = list1


all_of_df.drop('OBJECTID', axis=1, inplace=True)
all_of_df.drop('adm_cd', axis=1, inplace=True)
all_of_df.drop('adm_cd2', axis=1, inplace=True)
all_of_df.drop('sidonm', axis=1, inplace=True)
all_of_df.drop('sggnm', axis=1, inplace=True)
all_of_df.drop('기간', axis=1, inplace=True)
all_of_df.drop('자치구', axis=1, inplace=True)
all_of_df.drop('동', axis=1, inplace=True)
all_of_df.drop('주소', axis=1, inplace=True)

all_of_df['면적대비지하철개수'] = all_of_df['지하철개수'] / all_of_df['면적']
display(all_of_df)
all_of_df.to_excel('./data/pro_data/시험.xlsx')
# # 새로운 리스트를 통합정리 파일에 새로운 열로 추가한다.


In [ ]:
# 성희가 보내준 파일 불러와서 계산하기
import pandas as pd
all_of_df = pd.read_excel('./data/pro_data/통합정리1.xlsx')

all_of_df.drop('adm_cd', axis=1, inplace=True)
all_of_df.drop('adm_cd2', axis=1, inplace=True)
all_of_df.drop('sidonm', axis=1, inplace=True)
all_of_df.drop('sggnm', axis=1, inplace=True)
all_of_df.drop('기간', axis=1, inplace=True)
all_of_df.drop('자치구', axis=1, inplace=True)
all_of_df.drop('주소', axis=1, inplace=True)


count1 = pd.read_csv('./data/pro_data/지하철.csv')
count2 = pd.read_csv('./data/pro_data/버스정류장.csv')

# display(all_of_df)
# display(count1)
# display(count2)

# 지하철역 카운트
first_count = pd.merge(all_of_df, count1, how='outer', left_on='동', right_on='ADM_DR_NM_')
first_count.drop('ADM_DR_NM_', axis=1, inplace=True)
first_count.drop('Sum_Join_C', axis=1, inplace=True)
first_count.drop(424, axis=0, inplace=True)
first_count.rename(columns = {'Cnt_ADM_DR' : '지하철역개수'}, inplace=True)
first_count['지하철역개수'].fillna(0, inplace=True)
display(first_count)

check_count = pd.merge(first_count, count2, how='outer', left_on='동', right_on='ADM_DR_NM_')
check_count.drop('ADM_DR_NM_', axis=1, inplace=True)
check_count.drop('Sum_Join_C', axis=1, inplace=True)
check_count.rename(columns = {'Cnt_ADM_DR' : '버스정거장개수'}, inplace=True)
check_count.drop(424, axis=0, inplace=True)
display(check_count)

# check_count['면적대비지하철역'] = check_count['지하철역개수'] / check_count['면적']
# check_count['면적대비버스정류장'] = check_count['버스정거장개수'] / check_count['면적']
# display(check_count)

check_count['접근성'] = (check_count['지하철역개수']*4 + check_count['버스정거장개수']) / check_count['면적']
check_count['교통편'] = check_count['지하철역개수']*4 + check_count['버스정거장개수']
display(check_count)

check_count.to_excel('./data/pro_data/테스트.xlsx')



In [ ]:
## 특정 단어가 포함된 행 삭제하기

# # 동별 면적 불러오기 --> 면적 단위는 km^2 // 면적 이미 있음
# area = pd.read_csv('./data/서울시행정구역동별통계.txt', sep="\t")
# # 자치구, 동에 "합계", "소계"가 포함된 행 없애기
# area_1 = area['자치구'].isin(['합계','소계'])
# area_2 = area[~area_1]
# area_3 = area_2['동'].isin(['합계','소계'])

# area_last = area_2[~area_3]
# # display(area_last.head())


# all_data = pd.merge(pop_last, area_last, how='left', on='동')
# # display(all_data)

In [ ]:
# 동별 중앙값 불러오기
# dong = pd.read_csv('./data/pro_data/행정동별위치정보.csv', encoding='euc-kr')
# dong = dong[['행정동명', '경도', '위도']]
# dong.rename(columns = {'행정동명': '동'}, inplace=True)



# # # 동별 써클 그리기 --> 동별 중앙값을 찾은 뒤 거기서 반지름 50을 주자
# for dong,lat,lng in zip(all_data.동, all_data.위도, all_data.경도):
#     folium.CircleMarker(
#       [lat,lng],
#       radius=100,
#       color='#00ff00', # 테두리
#       fill_color='#ff000099', # 원 칼라 // 투명도도 같이 준다.
#       popup=dong
#     ).add_to(map)
#     folium.Marker(
#         [lat,lng],
#         icon=folium.Icon(color='red')
#     ).add_to(map)


In [ ]:
# 서울 동별 인구수, 면적정보, 중앙값 합치기
# all_data = pd.merge(pop_last, dong, how='outer', on='동')
# all_data.dropna(axis=0, how='any', inplace=True)

In [2]:
## 다른 사람 자료 확인해보기
import pandas as pd
juhee2 = pd.read_csv('./data/pro_data/dong_df_1110.csv')
juhee3 = pd.read_csv('./data/pro_data/자전거대여수_all.csv')

display(juhee2)
display(juhee3)

,동,생산인구,경사도,자전거도로길이,총거치대수,총대여수,비율
0,가락1동,22381.0,-15.5247,7150.0,115.0,74652.0,64914.78
1,가락2동,18686.0,-39.3115,4704.0,45.0,33583.0,74628.89
2,가락본동,25532.0,-30.7845,6932.0,55.0,25487.0,46340.00
3,가리봉동,7566.0,-21.8088,0.0,10.0,5621.0,56210.00
4,가산동,25926.0,-12.9490,2108.0,258.0,160033.0,62028.29
...,...,...,...,...,...,...,...
418,효창동,9982.0,-39.3094,0.0,35.0,18699.0,53425.71
419,후암동,10701.0,-110.1838,0.0,10.0,72.0,720.00
420,휘경1동,12058.0,-15.7920,1889.0,30.0,41753.0,139176.67
421,휘경2동,17543.0,-18.4596,3782.0,51.0,15335.0,30068.63


,대여소번호,대여소명,동,구,총거치대수,2020년_대여수,2019년_대여수
0,1695,월계미륭아파트 정문,월계3동,노원구,10,0,0
1,2301,현대고등학교 건너편,신사동,강남구,10,11352,17117
2,2302,교보타워 버스정류장(신논현역 3번출구 후면),논현1동,강남구,10,4762,12548
3,2303,논현역 7번출구,논현1동,강남구,15,2367,8413
4,2304,신영 ROYAL PALACE 앞,논현2동,강남구,10,931,2631
...,...,...,...,...,...,...,...
2078,1572,우이산 입구,우이동,강북구,20,0,0
2079,2547,포드서초서비스센터앞,잠원동,서초구,10,0,0
2080,1573,현대성우아파트 옆,송천동,강북구,10,0,0
2081,1268,몽촌토성역 1번출구 옆,오륜동,송파구,15,11444,24080


In [47]:
## 데이터파일 통합 정리 --> 내꺼하고 다른 사람 꺼
import pandas as pd
import re

data1 = pd.read_excel('./data/pro_data/최종정리.xlsx')
data2 = pd.read_excel('./data/pro_data/테스트.xlsx', thousands = ',')

# display(data1)
# display(data2)

all_of_data = pd.merge(data1, data2, how='outer', on='동')


all_of_data = all_of_data.drop(all_of_data.columns[[0,2,8,9]], axis=1)

all_of_data.info()
# display(all_of_data)

all_of_data['자전거이용비율'] = all_of_data['총대여수'] / all_of_data['인구']



all_of_data.to_excel('./data/pro_data/last.xlsx')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424 entries, 0 to 423
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   동        424 non-null    object 
 1   경사도      424 non-null    float64
 2   자전거도로길이  424 non-null    int64  
 3   총거치대수    406 non-null    float64
 4   총대여수     406 non-null    float64
 5   비율       406 non-null    float64
 6   인구       424 non-null    int64  
 7   면적       424 non-null    float64
 8   인구밀도     424 non-null    int64  
 9   지하철역개수   424 non-null    int64  
 10  버스정거장개수  424 non-null    int64  
 11  접근성      424 non-null    float64
 12  교통편      424 non-null    int64  
dtypes: float64(6), int64(6), object(1)
memory usage: 46.4+ KB
